In [1]:
from experiments import *

In [2]:
# inputlist = ["Helheim_Big_PINN_obs_ITSLIVE0"] #, "Helheim_Big_PINN_obs_ITSLIVE2007", "Helheim_Big_PINN_obs_Rignot2012", "Helheim_Big_PINN_obs_JoughinComposite"]
# outputlist = ["SSA2D_ITSLIVE0_4000"] #, "SSA2D_ITSLIVE2007", "SSA2D_Rignot2012", "SSA2D_Joughin"]

In [3]:
inputlist = ["Helheim_Weertman_iT080_PINN_fastflow_CF_1000"] 
outputlist = ["SSA2D_FlightTrackH500"] 

In [ ]:
for i in range(len(inputlist)):
    experiment_2D_3NN_test((5, 3, 5, 8, 14), epochADAM=1000000, seed=None, N_u=4000, N_H=500, N_C=None, N_f=9000, NNeurons=20, NLayers=6, FlightTrack=True, inputFileName=inputlist[i], outputFileName=outputlist[i])

Use 497 flight track data for the ice thickness training data
Hyperparameters:
{
  "N_u": 4000,
  "N_s": 4000,
  "N_H": 500,
  "FlightTrack": true,
  "N_C": null,
  "N_f": 9000,
  "layers": [
    2,
    20,
    20,
    20,
    20,
    20,
    20,
    2
  ],
  "h_layers": [
    2,
    20,
    20,
    20,
    20,
    20,
    20,
    2
  ],
  "C_layers": [
    2,
    20,
    20,
    20,
    20,
    20,
    20,
    1
  ],
  "tf_epochs": 1000000,
  "tf_lr": 0.001,
  "tf_b1": 0.99,
  "tf_eps": 0.1,
  "nt_epochs": 0,
  "log_frequency": 10000,
  "save_history": true,
  "history_frequency": 10
}

TensorFlow version: 2.4.1
Eager execution: True
GPU-accerelated: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training set: 

uv: (4000, 2) 

s: (4000, 1) 

H: (1039, 1) 

C: (542, 1) 


Training started
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
minmax_scale_layer (MinmaxS

In [ ]:
for i in range(len(inputlist)):
    experiment_2D_3NN_test((5, 3, 5, 10, 16), epochADAM=1000000, seed=None, N_u=4000, N_H=500, N_C=None, N_f=9000, NNeurons=20, NLayers=6, FlightTrack=True, inputFileName=inputlist[i], outputFileName=outputlist[i])